In [1]:
#### ------------------------------------------------------------------------------------------
#### author: Ranjan Barman, date: Feb 9, 2025
#### Compute 12NPIFs based on HoverNet prediction, using MPP = 0.248 for unit conversion
#### Removes outliers for Major Axis and Minor Axis using IQR
#### Filters top 25% tiles based on majority of cancer nuclei
#### ------------------------------------------------------------------------------------------

import os
import pandas as pd
import numpy as np

# Set working directory
_wpath_ = "/data/Lab_ruppin/Ranjan/HnE/"
os.makedirs(_wpath_, exist_ok=True)
os.chdir(_wpath_)

print("Working directory:", _wpath_)

# Define dataset name and output file path dynamically
dataset_name = "TCGA_BRCA_FFPE"
input_folder = f"{dataset_name}/outputs/HoverNet/"
output_file_path = f"{dataset_name}/outputs/HoverNet/HoverNet_NPIFs_TCGA_BRCA_1106_Filter_Top_25Q.csv"

# Define column names for computation
columns_to_compute = ["Area", "Major Axis", "Minor Axis", "Perimeter", "Eccentricity", "Circularity"]

# Define MPP for conversion
MPP = 0.248  

# Function to remove outliers using IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Store results in a list
results = []

# Get all folders matching TCGA pattern
tcga_folders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f)) and f.startswith("TCGA")]

# Process each detected TCGA folder
for slide_name in tcga_folders:
    file_path = os.path.join(input_folder, slide_name, "features", f"{slide_name}.csv")
    
    # Check if file exists before proceeding
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        continue

    # Load CSV file
    df = pd.read_csv(file_path)

    # Convert units using MPP where applicable
    df["Area"] = df["Area"] * (MPP ** 2)  # Convert area to µm²
    df["Major Axis"] = df["Major Axis"] * MPP  # Convert major axis length to µm
    df["Minor Axis"] = df["Minor Axis"] * MPP  # Convert minor axis length to µm
    df["Perimeter"] = df["Perimeter"] * MPP  # Convert perimeter to µm

    # Remove outliers for Major Axis and Minor Axis
    df = remove_outliers(df, "Major Axis")
    df = remove_outliers(df, "Minor Axis")

    # Count total nuclei per tile
    tile_nucleus_counts = df.groupby("Tile")["Nucleus ID"].count().reset_index()
    tile_nucleus_counts.rename(columns={"Nucleus ID": "Nucleus_Count"}, inplace=True)

    # Get total number of tiles with cancer nuclei
    total_tiles_with_nuclei = tile_nucleus_counts.shape[0]

    # Compute the 75th percentile (top 25%) nucleus count for filtering
    percentile_75 = tile_nucleus_counts["Nucleus_Count"].quantile(0.75)

    # Filter top 25% tiles with highest nucleus count
    top_25_percent_tiles = tile_nucleus_counts[tile_nucleus_counts["Nucleus_Count"] >= percentile_75]

    # Get the number of tiles that passed the filtering
    num_filtered_tiles = top_25_percent_tiles.shape[0]

    print(f"Slide: {slide_name}")
    print(f"  - Total tiles with cancer nuclei: {total_tiles_with_nuclei}")
    print(f"  - 75th percentile nucleus count threshold: {percentile_75}")
    print(f"  - Number of tiles selected (top 25%): {num_filtered_tiles}")

    # Merge filtered tiles with original dataframe
    df_filtered = df[df["Tile"].isin(top_25_percent_tiles["Tile"])]

    if df_filtered.empty:
        print(f"  - No tiles with nucleus count in the top 25% for {slide_name}, skipping...\n")
        continue

    # Compute mean and std for required columns
    mean_values = df_filtered[columns_to_compute].mean()
    std_values = df_filtered[columns_to_compute].std()

    # Append results
    results.append([slide_name, total_tiles_with_nuclei, num_filtered_tiles] + mean_values.tolist() + std_values.tolist())

# Create DataFrame for results
result_df = pd.DataFrame(
    results,
    columns=["Slide_Name", "Total_Tiles", "Filtered_Tiles"] + 
            [f"Mean {col}" for col in columns_to_compute] + 
            [f"Std {col}" for col in columns_to_compute]
)

# Handle NaN and infinite values: Replace them with column mean
result_df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Convert inf to NaN
result_df.fillna(result_df.mean(numeric_only=True), inplace=True)  # Replace NaN with column mean

# If a column is entirely NaN, replace with zero
result_df.fillna(0, inplace=True)

# Save results to CSV
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)  # Ensure directory exists
result_df.to_csv(output_file_path, index=False)

print(f"\nFiltered results saved to: {output_file_path}")


Working directory: /data/Lab_ruppin/Ranjan/HnE/
Converted results saved to: TCGA_BRCA_FFPE/outputs/HoverNet/HoverNet_NPIFs_Test5.csv


In [2]:
result_df

,Slide_Name,Mean Area,Mean Major Axis,Mean Minor Axis,Mean Perimeter,Mean Eccentricity,Mean Circularity,Std Area,Std Major Axis,Std Minor Axis,Std Perimeter,Std Eccentricity,Std Circularity
0,TCGA-LL-A440-01Z-00-DX1_757_tiles,12.409250,5.256150,3.372088,14.112739,0.693992,0.744568,6.482429,21.419228,1.094149,3.745886,0.149467,0.099584
1,TCGA-LL-A73Y-01Z-00-DX1_1223_tiles,7.907922,3.957031,2.721694,11.319506,0.656373,0.749095,5.173203,2.809882,0.779769,3.686583,0.160548,0.118691
2,TCGA-OL-A66K-01Z-00-DX1_9040_tiles,33.611253,8.345863,5.719371,22.575683,0.662051,0.774815,15.818132,19.902820,1.694468,5.523841,0.150711,0.095773
3,TCGA-LL-A7T0-01Z-00-DX1_5390_tiles,14.211773,43.997907,3.722092,14.826658,0.641318,0.762794,6.976532,38265.898249,1.094873,3.693708,0.160020,0.105056
4,TCGA-LL-A8F5-01Z-00-DX1_6391_tiles,17.080745,5.704066,3.980249,16.008476,0.653337,0.764940,11.021945,18.132684,1.369280,4.985657,0.155970,0.096943
